## Katse mõelda

In [1]:
from __future__ import absolute_import, division, print_function
from tensorflow.keras import Model, layers
import matplotlib.pyplot as plt
import tensorflow as tf
import numpy as np
from tensorflow.keras.datasets import mnist

rng = np.random

In [2]:
import os
# 2. DOWNLOAD THE DATASET
# Computer must have a unique kaggle.json file downloaded and have this path: "C:\Users\youruser\.kaggle\kaggle.json"
import kaggle

dataset_name = 'samithsachidanandan/human-face-emotions'
target_folder = 'human-face-emotions'

kaggle.api.dataset_download_files(dataset_name, path=target_folder, unzip=True)
print("Download complete!")

Dataset URL: https://www.kaggle.com/datasets/samithsachidanandan/human-face-emotions
Download complete!


## Loading and Preprocessing of Data

In [3]:
import tensorflow as tf

data_dir = './human-face-emotions/data' 


# Read data from directory containing separate directories for each label value - anger, fear, happiness, sadness, surprise
train_ds = tf.keras.preprocessing.image_dataset_from_directory(
    data_dir,
    validation_split=0.2,     # 80/20 split
    subset="training",
    seed=42,                  # ensures same split every run
    label_mode="int",
    image_size=(48, 48),      # 
    color_mode="grayscale",   # or "rgb"
    batch_size=None,          # important: gives Numpy-like tensors
    shuffle=True              # shuffle!!
)

test_ds = tf.keras.preprocessing.image_dataset_from_directory(
    data_dir,
    validation_split=0.2,
    subset="validation",
    seed=42,
    label_mode="int",
    image_size=(48, 48),
    color_mode="grayscale",
    batch_size=None # on iteration  gives one image at time
    # no shuffle!
)

class_names = train_ds.class_names
number_of_classes = 5

Found 59099 files belonging to 5 classes.
Using 47280 files for training.
Found 59099 files belonging to 5 classes.
Using 11819 files for validation.


In [4]:
train_ds = train_ds.shuffle(10000).batch(128).prefetch(tf.data.AUTOTUNE)
test_ds  = test_ds.batch(128).prefetch(tf.data.AUTOTUNE)

### Building the model

In [5]:
model = tf.keras.Sequential([

    # --- Convolution blocks ---
    tf.keras.layers.Conv2D(32, (3,3), activation='relu', padding='same', input_shape=(48, 48, 1)),
    tf.keras.layers.MaxPooling2D(),
    # 32: number of filters -> 32 different pattern detectors
    # (3×3) = kernel (=filter) size in pixels - like a (3×3) window
    # padding='same' : 48×48 input → 48×48 output
    # MaxPooling : reduces image size by taking the maximum value in each 3x3 window
    

    tf.keras.layers.Conv2D(64, (3,3), activation='relu', padding='same'),
    tf.keras.layers.MaxPooling2D(),

    tf.keras.layers.Conv2D(128, (3,3), activation='relu', padding='same'),
    tf.keras.layers.MaxPooling2D(),

    tf.keras.layers.Flatten(),

    # --- Dense layers ---
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dropout(0.3),
    tf.keras.layers.Dense(number_of_classes, activation='softmax')
])

c:\Users\mattisvr\AppData\Local\anaconda3\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:113: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [6]:
model.compile(
    optimizer=tf.keras.optimizers.Adam(1e-3),
    loss="sparse_categorical_crossentropy",
    metrics=["accuracy"]
)

model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d (Conv2D)                 │ (None, 48, 48, 32)     │           320 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d (MaxPooling2D)    │ (None, 24, 24, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_1 (Conv2D)               │ (None, 24, 24, 64)     │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_1 (MaxPooling2D)  │ (None, 12, 12, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_2 (Conv2D)               │ (None, 12, 12, 128)    │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_2 (MaxPooling2D)  │ (None, 6, 6, 128)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 4608)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 128)            │       589,952 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 5)              │           645 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 683,269 (2.61 MB)

 Trainable params: 683,269 (2.61 MB)

 Non-trainable params: 0 (0.00 B)

In [7]:
history = model.fit(
    train_ds,
    validation_data=test_ds,
    epochs=6
)

Epoch 1/6
370/370 ━━━━━━━━━━━━━━━━━━━━ 339s 769ms/step - accuracy: 0.3174 - loss: 1.8380 - val_accuracy: 0.3864 - val_loss: 1.4240
Epoch 2/6
370/370 ━━━━━━━━━━━━━━━━━━━━ 91s 220ms/step - accuracy: 0.4021 - loss: 1.3949 - val_accuracy: 0.4805 - val_loss: 1.2967
Epoch 3/6
370/370 ━━━━━━━━━━━━━━━━━━━━ 89s 217ms/step - accuracy: 0.4537 - loss: 1.3116 - val_accuracy: 0.5250 - val_loss: 1.2162
Epoch 4/6
370/370 ━━━━━━━━━━━━━━━━━━━━ 88s 214ms/step - accuracy: 0.5073 - loss: 1.2118 - val_accuracy: 0.5608 - val_loss: 1.0960
Epoch 5/6
370/370 ━━━━━━━━━━━━━━━━━━━━ 87s 213ms/step - accuracy: 0.5569 - loss: 1.0844 - val_accuracy: 0.5891 - val_loss: 1.0136
Epoch 6/6
370/370 ━━━━━━━━━━━━━━━━━━━━ 96s 233ms/step - accuracy: 0.5936 - loss: 0.9959 - val_accuracy: 0.6131 - val_loss: 0.9809
